In [ ]:
# Does not need to be executed if
# ~/.ipython/profile_default/ipython_config.py
# exists and contains:
# get_config().InteractiveShell.ast_node_interactivity = 'all'

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
from collections import defaultdict, deque
from copy import copy, deepcopy
from itertools import islice
from pprint import pprint

A _definite logic program_ consists of _definite clauses_, that is, _universal closures_ of implications whose right hand sides are _atomic formulas_ and whose left hand sides are conjunctions of atomic formulas. Because a conjunction over an empty set of formulas is logically true, and an implication whose left hand side is a tautology is logically equivalent to its right hand side, atomic formulas, also known as _atoms_ or _facts_, are particular cases of definite clauses. Here is a definite logic program consisting of 13 definite clauses, the first 8 of which are facts:  

$\mathrm{father(bob,jack)}$  
$\mathrm{father(bob,sandra)}$  
$\mathrm{father(john,bob)}$  
$\mathrm{father(john,mary)}$  
$\mathrm{mother(jane,jack)}$  
$\mathrm{mother(jane,sandra)}$  
$\mathrm{mother(emily,bob)}$  
$\mathrm{mother(emily,mary)}$  
$\mathrm{\forall X\forall Y\bigl(father(X,Y)\rightarrow parent(X,Y)\bigr)}$  
$\mathrm{\forall X\forall Y\bigl(mother(X,Y)\rightarrow parent(X,Y)\bigr)}$  
$\mathrm{\forall X\forall Y\bigl(parent(Y,X)\wedge male(X)\rightarrow son(X,Y)\bigr)}$  
$\mathrm{\forall X\forall Y\bigl(parent(Y,X)\wedge female(X)\rightarrow daughter(X,Y)\bigr)}$  
$\mathrm{\forall X\forall Y\forall Z\bigl(male(X)\wedge parent(Z,X)\wedge parent(Z,Y)\rightarrow brother(X,Y)\bigr)}$  
$\mathrm{\forall X\forall Y\forall Z\bigl(parent(X,Z)\wedge parent(Z,Y)\rightarrow grandparent(X,Y)\bigr)}$

In English, this reads as:

* Bob is a father of Jack and Sandra, John is a father of Bob and Mary, Jane is a mother of Jack and Sandra, Emily is a mother of Bob and Mary.
* For all X, for all Y, if X is a father of Y then X is a parent of Y.
* For all X, for all Y, if X is a mother of Y then X is a parent of Y.
* For all X, for all Y, if Y is a parent of X and X is male then X is a son of Y.
* For all X, for all Y, if Y is a parent of X and X is female then X is a daughter of Y.
* For all X, for all Y, for all Z, if X is male, Z is a parent of X and Z is a parent of Y then X is is a brother of Y.
* For all X, for all Y, for all Z, if X is a parent of Z and Z is a parent of Y, then X is a grandparent of Y.

Note that the last two definite clauses are logically equivalent to:

$\mathrm{\forall X\forall Y\Bigl(male(X)\wedge\exists Z\bigl(parent(Z,X)\wedge parent(Z,Y)\bigr)\rightarrow brother(X,Y)\Bigr)}$  
$\mathrm{\forall X\forall Y\Bigl(\exists Z\bigl(parent(X,Z)\wedge parent(Z,Y)\bigr)\rightarrow grandparent(X,Y)\Bigr)}$

which reads as:

* For all X, for all Y, if X is male and there exists Z such that Z is a parent of X and Z is a parent of Y, then X is is a brother of Y.
* For all X, for all Y, if there exists Z such that X is a parent of Z and Z is a parent of Y, then X is a grandparent of Y.

In Prolog, this definite logic program takes the form:

father(bob, jack).  
father(bob, sandra).  
father(john, bob).  
father(john, mary).  
mother(jane, jack).  
mother(jane, sandra).  
mother(emily, bob).  
mother(emily, mary).  
parent(X, Y) :- father(X, Y).  
parent(X, Y) :- mother(X, Y).  
son(X, Y) :- parent(Y, X), male(X).   
daughter(X, Y) :- parent(Y, X), female(X).  
brother(X, Y) :- male(X), parent(Z, X), parent(Z, Y).  
grandparent(X, Y) :- parent(X, Z), parent(Z, Y).

An English reading of the definite clauses that are not facts that more closely follows this alternative syntax is:

* For all X, for all Y, for X to be a parent of Y, it suffices that X be a father of Y.
* For all X, for all Y, for X to be a parent of Y, it suffices that X be a mother of Y.
* For all X, for all Y, for X to be a son of Y, it suffices that Y be a parent of X and that X be male.
* For all X, for all Y, for X to be a daughter of Y, it suffices that Y be a parent of X and that X be female.
* For all X, for all Y, for X to be a brother of Y, it suffices that X be male and that some Z exists that is a parent of both X and Y.
* For all X, for all Y, for X to be a grandparent of Y, it suffices that some Z exists such that X is a parent of Z and Z is a parent of Y.

The logical syntax suggests successive applications of _instantiation_ and _modus ponens_ to derive some atoms such as $\mathrm{grandparent(john,jack)}$ from the given facts, in a _bottom up_ manner:

* It is known that $\mathrm{father(bob,jack)}$. Together with $\mathrm{\forall X\forall Y\bigl(father(X,Y)\rightarrow parent(X,Y)\bigr)}$, this implies that $\mathrm{parent(bob,jack)}$.
* It is known that $\mathrm{father(john,bob)}$. Together with $\mathrm{\forall X\forall Y\bigl(father(X,Y)\rightarrow parent(X,Y)\bigr)}$, this implies that $\mathrm{parent(john,bob)}$.
* From $\mathrm{\forall X\forall Y\forall Z\bigl(parent(X,Z)\wedge parent(Z,Y)\rightarrow grandparent(X,Y)\bigr)}$ together with $\mathrm{parent(john,bob)}$ and $\mathrm{parent(bob,jack)}$, we infer that $\mathrm{grandparent(john,jack)}$, ending the proof.

The Prolog syntax suggests proving instances of some atomic formulas such as grandparent(john, jack) by proving instances of other atomic formulas that directly imply the latter, until nothing but given facts are eventually reached, in a _top down_ manner:

* To prove grandparent(john, jack), it suffices to find a value for Z and prove both parent(john, Z) and parent(Z, jack).
  * To find a value for Z and prove both parent(john, Z) and parent(Z, jack), it suffices to find a value for Z and either prove both father(john, Z) and parent(Z, jack) or prove both mother(john, Z) and parent(Z, jack).
    * To find a value for Z and prove both father(john, Z) and parent(Z, jack), one can let Z be either bob or mary as indeed father(john, bob) and father(john, mary) are given, having to then prove either parent(bob, jack) or parent(mary, jack).
      * To prove parent(bob, jack), it suffices to prove either father(bob, jack) or mother(bob, jack).
        * Indeed, father(bob, jack) is given, ending the proof.
         
Note that the definite clauses that could be used were selected in the order in which they appeared. Suppose for instance that the clauses of the definite logic program that are not facts were listed as follows in Prolog form:

parent(X, Y) :- mother(X, Y).  
parent(X, Y) :- father(X, Y).  
daughter(X, Y) :- parent(Y, X), female(X).  
son(X, Y) :- parent(Y, X), male(X).   
brother(X, Y) :- male(X), parent(Z, X), parent(Z, Y).  
grandparent(X, Y) :- parent(X, Z), parent(Z, Y).

Then the proof of grandparent(john, jack) would proceed as follows:

* To prove grandparent(john, jack), it suffices to find a value for Z and prove both parent(john, Z) and parent(Z, jack).
  * To find a value for Z and prove both parent(john, Z) and parent(Z, jack), it suffices to find a value for Z and either prove both mother(john, Z) and parent(Z, jack) or prove both father(john, Z) and parent(Z, jack).
    * One cannot find a value for Z and prove mother(john, Z).
    * To find a value for Z and prove both father(john, Z) and parent(Z, jack), one can let Z be either bob or mary as indeed father(john, bob) and father(john, mary) are given, having to then prove either parent(bob, jack) or parent(mary, jack).
      * To prove parent(bob, jack), it suffices to prove either mother(bob, jack) or father(bob, jack).
        * mother(bob, jack) cannot be proved.
        * Indeed, father(bob, jack) is given, ending the proof.

grandparent(john, jack) is a _closed_ formula: it contains no variable. Working with formulas that do contain variables, the approach that has been described lets us do more than prove: it lets us compute. Rather than talking about proving an atomic formula, one talks about _solving a goal_ or _answering a query_. For instance, solving the goal grandparent(john, X) lets us compute the instantiations of X that make the resulting formula a logical consequence of the logic program in its logical form. With the original ordering of the clauses, this proceeds as follows:

* To solve grandparent(john, X), it suffices to find values for Z and Y and solve both parent(john, Z) and parent(Z, Y). One can then give X the value of Y.
  * To find values for Z and Y and solve both parent(john, Z) and parent(Z, Y), it suffices to find values for Y and Y_0 and either solve both father(john, Y_0) and parent(Y_0, Y) or solve both mother(john, Y_0) and parent(Y_0, Y). One can then give X the value of Y.
    * To find values for Y and Y_0 and solve both father(john, Y_0) and parent(Y_0, Y), one can let Y_0 be either bob or mary as indeed father(john, bob) and father(john, mary) are given, having to then solve either parent(bob, Y) or parent(mary, Y). One can then give X the value of Y.
      * To find a value for Y and solve parent(bob, Y), it suffices to find a value for Y_0 and solve either father(bob, Y_0) or mother(bob, Y_0). One can then give X the value of Y_0.
        * To find a value for Y_0 and solve father(bob, Y_0), one can let Y_0 be either jack or sandra. This yields two solutions to the original goal: X can take the value jack, or it can take the value sandra.
        * One cannot find a value for Y_0 and solve mother(bob, Y_0).
      * To find a value for Y and solve parent(mary, Y), it suffices to find a value for Y_0 and solve either father(mary, Y_0) or mother(mary, Y_0). One can then give X the value of Y_0.
        * One cannot find a value for Y_0 and solve father(mary, Y_0).
        * One cannot find a value for Y_0 and solve mother(mary, Y_0).
  * One cannot find a value for Y_0 and solve mother(john, Y_0).

Hence there are two and only two solutions to the goal grandparent(john,X): X equal to jack, and X equal to sandra.

There is a bit of mystery in the preceding description in the way some variables are changed or new variables are introduced. This has to do with the fact that different occurrences of a given variable can be unrelated, which requires to sometimes rename variables and introduces somehow tricky technicalities:

* Observe that the X in the goal grandparent(john, X) is unrelated to the X in the _rule_ (a more usual name for "definite clause" with Prolog notation) grandparent(X, Y) :- parent(X, Z), parent(Z, Y). If we want to let the goal interact with the rule, it is safe to rename all occurrences of X in the rule, using a new variable, say X_0: grandparent(X_0, Y) :- parent(X_0, Z), parent(Z, Y). We can then _unify_ the goal grandparent(john, X) and the _head_ of grandparent(X_0, Y) :- parent(X_0, Z), parent(Z, Y), namely, grandparent(X_0, Y), by replacing X_0 in grandparent(X_0, Y) by john and replacing X in grandparent(john, X) by Y (we could as well replace Y in grandparent(X_0, Y) by X, but the code to be developed later opts for the first alternative). Then the same replacements can be performed in the _body_ of grandparent(X_0, Y) :- parent(X_0, Z), parent(Z, Y), namely, parent(X_0, Z), parent(Z, Y), yielding the rule grandparent(john, Y) :- parent(john, Z), parent(Z, Y). This allows one to replace the goal grandparent(john, X) with the goals parent(john, Z) and parent(Z, Y), knowing that the values of Y that yield solutions to those goals are values of X that yield solutions to the original goal.
* Observe that the Y in the goal parent(Z, Y), one of the two goals that we now have to solve, is unrelated to the Y in the rule parent(X, Y) :- father(X, Y). Also, the X in parent(X, Y) :- father(X, Y) is unrelated to the X in the original goal (grandparent(john, X)). If we want to let the other goal we now have to solve, parent(john, Z), interact with that rule, and let the X in the original goal alone, it is safe to rename all occurrences of Y in the rule, using a new variable, say Y_0, and rename all occurrences of X in the rule using a new variable, say X_0: parent(X_0, Y_0) :- father(X_0, Y_0). We can then unify the goal parent(john, Z) and the head of parent(X_0, Y_0) :- father(X_0, Y_0), by replacing X_0 in parent(X_0, Y_0) by john and replacing Z in parent(john, Z) by Y_0. Then the same replacements can be performed in the body of parent(X_0, Y_0) :- father(X_0, Y_0), yielding the rule parent(john, Y_0) :- father(john, Y_0). This allows one to replace the goal parent(john, Z) with the goal father(john, Y_0). But the Z in the goals parent(john, Z) and parent(Z, Y) are related, hence having replaced Z by Y_0 in parent(john, Z), we should replace Z by Y_0 in parent(Z, Y). So we goals we now have to solve are father(john, Y_0) and parent(Y_0, Y).

The description of the search for solutions to the goal grandparent(john, X) has the structure of a tree. The following graphical representation of the description makes it particularly clear:

![](tree_1.pdf)

Observe that with the alternative listing considered above of the clauses of the definite logic program that are not facts, the description of the search for solutions to the goal grandparent(john, X) would be captured by the following tree:

![](tree_2.pdf)

Instead of starting with one goal, one can start with a sequence of (implicitly conjuncted) goals. For instance, if the logic program was extended with the fact female(sandra), then there would be two solutions to the goals grandparent(john, X), daughter(X, Y): X equal to sandra and Y equal to bob, and X equal to sandra and Y equal to jane.

The atoms considered up to now are built from _predicate symbols_ (father, mother, parent, grandparent, male, female, son, daughter, brother), all of _arity_ 2, that is, all _binary_ predicate symbols, _constants_ (bob, jack, sandra, john, mary, jane, emily), and variables. Constant and variables are called _terms_. If we introduce _function symbols_ (of strictly positive arity, as constants are nothing but function symbols of arity 0, or _nullary_ function symbols), then we can build more complex terms. For instance, to represent lists of 0's and 1's, we can use:

* 3 constants, o for 0, i for 1, and e for the empty list;
* a binary function symbol (function symbol of arity 2) l, whose first argument is meant o or i and represent the first element of a nonempty list, and whose second argument is meant to be a term representing the rest of the list.

For instance:

* the term l(i, e) represents the list [1].
* the term l(o, l(i, e)) represents the list [0, 1].
* the term l(o, l(o, l(i, e))) represents the list [0, 0, 1].
* the term l(i, l(o, l(o, l(i, e)))) represents the list [1, 0, 0, 1].
* the term l(i, l(i, l(o, l(o, l(i, e))))) represents the list [1, 1, 0, 0, 1].

Terms such as l(e, e), l(i, o), l(l(i, o), e) and l(l(i, e), l(i, e)) are syntactically valid, but are given no interpretation. 

We can then consider the logic program that defines the join (concatenation) function:

join(e, X, X).  
join(l(H, T), X, l(H, Y)) :- join(T, X, Y).

This reads as:

* Joining the empty list and a list X results in X itself.
* To join a nonempty list and a list X, it suffices to join T and X, with T the list consisting of all elements of the first list except the first element H, resulting in a list Y, and put H at the front.

This allows us to solve a goal such as join(l(i, l(i, l(o, e))), l(o, l(i, e)), X), with as unique solution X equal to l(i, l(i, l(o, l(o, l(i, e))))), reflecting the fact that joining [1, 1, 0] and [0, 1] results in the list [1, 1, 0, 0, 1]. But one can also solve a goal such as join(X, X, Y), so look for all possible ways to join a list with (a copy of) itself. There are infinitely many solutions, for instance:

* X = e and Y = e: joining the empty list with itself results in the empty list.
* X = l(o, e) and Y = l(o, l(o, e)): joining [0] with itself results in [0, 0].
* X = l(i, e) and Y = l(i, l(i, e)): joining [1] with itself results in [1, 1].
* X = l(i, l(o, e)) and Y = l(i, l(o, l(i, l(o, e)))): joining [1, 0] with itself results in [1, 0, 1, 0].

Though they are correct, these solutions are not the most general ones, with the exception of the first one: all others are instances of more general solutions, that themselves contain variables:

* X equal to l(H, e) and Y equal to l(H, l(H, e)): joining a list of the form [H] (where H is equal to 0 or 1) with itself results in the list [H, H].
* X equal to l(H, l(H_0, e)) and Y equal to l(H, l(H_0, l(H, l(H_0, e)))): joining a list of the form [H, H_0] (where H and H_0 are independently equal to 0 or 1) with itself results in the list [H, H_0, H, H_0].

By computing _most general unifiers_, one guarantees that nothing but most general solutions be generated. To solve the goal join(X, X, Y):

* We first consider the fact join(e, X, X), change it to join(e, X_0, X_0), unifies join(X, X, Y) and join(e, X_0, X_0) with the equalities X = e, X_0 = X and Y = X_0, from which we derive X = e, X_0 = e and Y = e, which yields the solution X = e and Y = e.
* We then consider the rule join(l(H, T), X, l(H, Y)) :- join(T, X, Y), change it to join(l(H, T), X_0, l(H, Y_0)) :- join(T, X_0, Y_0), unifies join(X, X, Y) and join(l(H, T), X_0, l(H, Y_0)) with the equalities X = l(H, T), X_0 = X and Y = l(H, Y_0), from which we derive X = l(H, T), X_0 = l(H, T) and Y = l(H, Y_0), having to now solve the goal join(T, l(H, T), Y_0).
  * We first consider the fact join(e, X, X), change it to join(e, X_0, X_0), unifies join(T, l(H, T), Y_0) and join(e, X_0, X_0) with the equalities T = e, X_0 = l(H, T) and Y_0 = X_0, from which we derive T = e, X_0 = l(H, e) and Y_0 = l(H, e), which together with X = l(H, T) and Y = l(H, Y_0), yields the solution X = l(H, e) and Y = l(H, l(H, e)).
  * We then consider the rule join(l(H, T), X, l(H, Y)) :- join(T, X, Y), change it to join(l(H_0, T_0), X_0, l(H_0, Y_1)) :- join(T_0, X_0, Y_1), unifies join(T, l(H, T), Y_0) and join(l(H_0, T_0), X_0, l(H_0, Y_1)) with the equalities T = l(H_0, T_0), X_0 = l(H, T) and Y_0 = l(H_0, Y_1), from which we derive T = l(H_0, T_0), X_0 = l(H, l(H_0, T_0)), Y_0 = l(H_0, Y_1), X = l(H, l(H_0, T_0)) and Y = l(H, l(H_0, Y_1)), having to now solve the goal join(T_0, l(H, l(H_0, T_0)), Y_1).
    * We first consider the fact join(e, X, X), change it to join(e, X_0, X_0), unifies join(T_0, l(H, l(H_0, T_0)), Y_1) and join(e, X_0, X_0) with the equalities T_0 = e, X_0 = l(H, l(H_0, T_0)) and Y_1 = X_0, from which we derive T_0 = e, X_0 = l(H, l(H_0, e)) and Y_1 = l(H, l(H_0, e)), which together with X = l(H, l(H_0, T_0)) and Y = l(H, l(H_0, Y_1)), yields the solution X = l(H, l(H_0, e)) and Y = l(H, l(H_0, l(H, l(H_0, e)))).
    * We then consider the rule join(l(H, T), X, l(H, Y)) :- join(T, X, Y), change it to join(l(H_1, T), X_1, l(H_1, Y_0)) :- join(T, X_1, Y_0), unifies join(T_0, l(H, l(H_0, T_0)), Y_1) and join(l(H_1, T), X_1, l(H_1, Y_0)) with the equalities T_0 = l(H_1, T), X_1 = l(H, l(H_0, T_0)) and Y_1 = l(H_1, Y_0), from which we derive T_0 = l(H_1, T), X_1 = l(H, l(H_0, l(H_1, T))), Y_1 = l(H_1, Y_0), X = l(H, l(H_0, l(H_1, T))) and Y = l(H, l(H_0, l(H_1, Y_0))), having to now solve the goal join(T, l(H, l(H_0, l(H_1, T))), Y_0).
      * ...

The search tree is infinite and looks as follows:

![](tree_3.pdf)

Observe that if the clauses that define the join function were listed in reverse order, that is, as

join(l(H, T), X, l(H, Y)) :- join(T, X, Y).  
join(e, X, X).

then the search tree would look as follows:

![](tree_4.pdf)

So solving goals relative to a given definite logic program amounts to exploring a tree of the kind illustrated above; in fact, it is discovering, building the tree, and reporting solutions when reaching a _leaf_ associated with a solution. Two fundamental ways of exploring, or discovering, or building a tree are:

* __Depth-first__: the leftmost unexplored _branch_ is explored all the way down, _backtracking_ up to the first embranchment where new branches have still not been explored.
* __Breadth-first__: the _nodes_ are visited _level_ by level, from left to right on a given level.

To illustrate and experiment, let us consider the tree below:

![](tree_5.pdf)

The `defaultdict` class from the `collections` module offers an elegant way to represent a tree. The code in the following cell works as follows.

* `t = tree()` makes `t` denote a `defaultdict` object, say $t$.
* `t['A']['B']['F'] = None` executes as follows. An attempt is made to access the key `'A'` of $t$, which does not exist and is therefore created, with as value what `tree()`, returns, namely, a new `defaultdict` object, say $t_1$. So `t['A']` evaluates to $t_1$. An attempt is made to access the key `'B'` of $t_1$, which does not exist and is therefore created, with as value what `tree()`, returns, namely, a new `defaultdict` object, say $t_2$. So `t['A']['B']` evaluates to $t_2$. An attempt is made to access the key `'F'` of $t_2$, which does not exist and is therefore created, with as value what `tree()`, returns, namely, a new `defaultdict` object, say $t_3$. So `t['A']['B']['C']` evaluates to $t_3$ and is then changed to `None`.
* `t['A']['C']['G'] = None` executes as follows. An attempt is made to access the key `'A'` of $t$, which exists and  with `t['A']` evaluating to $t_1$. An attempt is made to access the key `'C'` of $t_1$, which does not exist and is therefore created and becomes the second key of $t_1$, with as value what `tree()`, returns, namely, a new `defaultdict` object, say $t_4$. So `t['A']['C']` evaluates to $t_4$. An attempt is made to access the key `'G'` of $t_4$, which does not exist and is therefore created, with as value what `tree()`, returns, namely, a new `defaultdict` object, say $t_5$. So `t['A']['C']['G']` evaluates to $t_5$ and is then changed to `None`.
* ...

The `pprint()` function from the `pprint` module makes it easier to see that `t` indeeds models the tree as intended:

In [ ]:
def tree():
    return defaultdict(tree)

t = tree()
t['A']['B']['F'] = None
t['A']['C']['G'] = None
t['A']['C']['H']['K'] = None
t['A']['C']['H']['L']['R'] = None
t['A']['C']['H']['M'] = None
t['A']['D'] = None
t['A']['E']['I']['N'] = None
t['A']['E']['I']['O']['S'] = None
t['A']['E']['J']['P']['T'] = None
t['A']['E']['J']['Q'] = None

pprint(t)

With this approach, a tree $t$ is modeled as a dictionary with a unique key, namely, the label of $t$'s __root__ (with `t` above as example, `'A'`), with as associated value, a dictionary that has as many keys as $t$'s root has __children__. That dictionary can be thought of as modeling a __forest__, namely, the collection of each __subtree__ of $t$ that has as root a child of $t$'s root (with `t` above as example, the subtrees rooted at `'B'`, `'C'`, `'D'` and `'E'`).

A recursive function makes it easy to explore a tree and list all nodes in a depth-first manner:

In [ ]:
def recursively_list_nodes_depth_first(t):
    if t is None:
        return
    for node in t:
        print(node, end=' ')
        recursively_list_nodes_depth_first(t[node])
        
recursively_list_nodes_depth_first(t)

It is easy to list the __paths__ from the root of the tree in a depth-first manner, so output [A], [A, B], [A, B, F], [A, C], [A, C, G], [A, C, H]... rather than A, B, F, C, G, H..., with the help of a __stack__. It is easy to list either the nodes or the paths from the root of the tree in a breadth-first manner with the help of a __queue__. Stacks are queues are essentially lists with a limited set of methods:

* For stacks, elements can be added and removed at one end (as plates brought to and removed from the top of a stack of plates, the sequence being viewed vertically rather than horizontally, with the end where the action takes place at the top). 
* For queues, elements can be added at one end and removed at the other end (as individuals queueing at a bus stop,   joining the queue at its back and leaving it, boarding the bus, at its front).

    Python lists with their `append()` and `pop()` methods offer suitable implementations of stacks, as the time complexity of both operations is constant in amortised cost. On the other hand, Python lists do not offer an effective implementation of queues: removing the first element of a list and inserting an element at the beginning of a list both have time complexity that is linear in the length of the list. The `deque` class from the `collections` module combines the functionality of stacks and queues, as it has methods for adding and removing elements at boths end that all have constant time complexity (thanks to a data structure known as a __doubly linked list__). Let us first use a `dequeue` object as a stack:

In [ ]:
# Alternatively: stack = deque([])
stack = deque(); stack
stack.append(0); stack
stack.append(1); stack
stack.append(2); stack
stack.pop(); stack # Two outputs
stack.append(3); stack
stack.append(4); stack
stack.pop(); stack # Two outputs
stack.pop(); stack # Two outputs
stack.pop(); stack # Two outputs

We can let a `deque` object $O$ model a queue in two ways, depending on how we match the ends of the queue with the ends of $O$. We can let the end of $O$ correspond to the front of the queue:

In [ ]:
# Alternatively: queue = deque([])
queue = deque(); queue
queue.appendleft(0); queue
queue.appendleft(-1); queue
queue.appendleft(-2); queue
queue.pop(); queue # Two outputs
queue.appendleft(-3); queue
queue.appendleft(-4); queue
queue.pop(); queue # Two outputs
queue.pop(); queue # Two outputs
queue.pop(); queue # Two outputs

Or we can let the end of the `deque` object correspond to the back of the queue:

In [ ]:
# Alternatively: queue = deque([])
queue = deque(); queue
queue.append(0); queue
queue.append(1); queue
queue.append(2); queue
queue.popleft(); queue # Two outputs
queue.append(3); queue
queue.append(4); queue
queue.popleft(); queue # Two outputs
queue.popleft(); queue # Two outputs
queue.popleft(); queue # Two outputs

Rather than appending elements to the back of a queue one after the other, we can prefer to in one sweep move, extend the back of the queue with all elements to append. In case the end of a `deque` object corresponds to the back of the queue it models, then the `extend()` method expectedly does the job. But in case the beginning of a `deque` object corresponds to the back of the queue, then the `extendleft()` method appropriately does the job too. Note how in the following cell, intending to append -1, then -2, then -3 on the left hand side, -3 indeed becomes the leftmost element, with -2 to its right, and -1 to the right of -2:

In [ ]:
queue = deque([0])
queue.extend([1, 2, 3])
queue.extendleft([-1, -2, -3])
queue

Writing a recursive function that explores a tree and lists all nodes in a depth-first manner was easy because behind the scene, a stack manages all recursive calls (more generally, a stack manages all function calls). Let us now perform that exploration without taking advantage of recursion; instead, let us explicitly use a stack. Considering the dictionary `t` that models the tree $t$ as defined above, `iter(t)` creates an iterable that can yield all of `t`'s keys. Actually, `t` has only one key, namely, the label of $t$'s root (`'A'`), which can be generated with `next(iter(t))`; let `root` denote it. Then `t[root]` is a dictionary that has as many keys as $t$'s root has children (namely 4, labeled `'B'`, `'C'`, `'D'` and `'E'`). Since the keys of `t[root]` have been inserted into the dictionary starting with the label of the leftmost child of $t$'s root (`'B'`), and proceeding from left to right all the way to the label of the rightmost child of $t$'s root (`'E'`), `reversed(list(t[root]))` is an iterable that can yield those labels starting with the rightmost child of $t$'s root and ending with the leftmost child of $t$'s root (so in the order `'E'`, `'D'`, `'C'` and `'B'`). Adding to the top of a stack `(k, t[root][k])` with `k` yielded by `reversed(list(t[root]))`, we eventually get in the stack a pair of the form $(l_1, f_1)$ where $l_1$ is the label (`'E'`) of the rightmost child of $t$'s root and $f_1$ is the forest consisting of the trees rooted at that node (so the trees rooted at the nodes labeled `'I'` and `'J'`), and above in the stack a pair of the form $(l_2, f_2)$ where $l_2$ is the label (`'D'`) of the second rightmost child of $t$'s root and $f_2$ is an empty forest since that child has no child, and above in the stack a pair of the form $(l_3, f_3)$ where $l_3$ is the label (`'C'`) of the third rightmost child of $t$'s root and $f_3$ is the forest consisting of the trees rooted at that node (so the trees rooted at the nodes labeled `'G'` and `'H'`), and at the top of the stack a pair of the form $(l_4, f_4)$ where $l_4$ is the label (`'B'`) of the leftmost child of $t$'s root and $f_4$ is the forest consisting of the trees rooted at that node (so only one tree, rooted at the node labeled `'F'`). The last pair is indeed the pair that we want to pop first from the stack, since when exploring $t$ in a depth-first manner, the nodes in $f_4$ are enumerated before the nodes in $f_3$, that are enumerated before the nodes in $f_2$, that are enumerated before the nodes in $f_1$. The next cell implements a function that explores a tree depth-first search and calls it with `t` passed as argument. The cell is followed with a cell that traces execution of that function call, replacing the forests stored in the stack with the roots of their trees:

In [ ]:
def list_nodes_depth_first(t):
    root = next(iter(t))
    roots_and_forests = deque([(root, t[root])])
    while roots_and_forests:
        root, forest = roots_and_forests.pop()
        print(root, end=' ')
        if forest:
            roots_and_forests.extend((k, forest[k])
                                          for k in reversed(list(forest))
                                    )

list_nodes_depth_first(t)

In [ ]:
root = next(iter(t))
roots_and_forests = deque([(root, t[root])])
print('Stack now (with trees changed to roots):\n     ',
      [(root, [k for k in t[root]])]
     )
while roots_and_forests:
    root, forest = roots_and_forests.pop()
    print()
    if forest:
        print('Node output:', root, '\t\tRoots of forest to process:',
              [k for k in forest]
             )
    else:
        print('Node output:', root, '\t\tEmpty forest')
    if forest:
        roots_and_forests.extend((k, forest[k])
                                      for k in reversed(list(forest))
                                )
    print('Stack now (with trees changed to roots):\n     ',
          [(root, [k for k in forest] if forest else None)
                 for (root, forest) in roots_and_forests
          ]
         )

It is easy to modify the previous function to enumerate paths from the root of the tree rather than nodes: instead of storing nodes, we store paths, starting with the path that starts from and stops at the root of the tree (`['A']`) and creating extensions of a path $p$ with each of the children of the last node in $p$, unless that node is a leaf:

In [ ]:
def list_paths_depth_first(t):
    root = next(iter(t))
    paths_and_forests = deque([([root], t[root])])
    while paths_and_forests:
        path, forest = paths_and_forests.pop()
        print(path)
        if forest:
            paths_and_forests.extend((path + [k], forest[k])
                                          for k in reversed(list(forest))
                                    )
        
list_paths_depth_first(t)

To explore a tree in a breadth-first manner and generate either nodes or paths, it suffices to modify the previous two functions, using a queue rather than a stack. Also, the left to right ordering of children of a given node should not be reversed. More precisely, the comments for `list_nodes_depth_first()` can be modified as follows (with `t` still denoting the dictionary that models the tree $t$ as defined above and `root` still denoting `next(iter(t))`, that is, the label of $t$'s root (`'A'`)). Adding to the back of a queue `(k, t[root][k])` with `k` yielded by `iter(t[root])`, we eventually get in the queue a pair of the form $(l_1, f_1)$ where $l_1$ is the label (`'B'`) of the leftmost child of $t$'s root and $f_1$ is the forest consisting of the trees rooted at that node (so only one tree, rooted at the node labeled `'F'`), and before in the queue a pair of the form $(l_2, f_2)$ where $l_2$ is the label (`'C'`) of the second lefttmost child of $t$'s root and $f_2$ is the forest consisting of the trees rooted at that node (so the trees rooted at the nodes labeled `'G'` and `'H'`), and before in the queue a pair of the form $(l_3, f_3)$ where $l_3$ is the label (`'D'`) of the third leftmost child of $t$'s root and $f_3$ is an empty forest since that child has no child, and at the end of the queue a pair of the form $(l_4, f_4)$ where $l_4$ is the label (`'E'`) of the rightmost child of $t$'s root and $f_4$ is the forest consisting of the trees rooted at that node (so the trees rooted at the nodes labeled `'I'` and `'J'`). The first pair is indeed the pair that we want to come to the front of the queue and be removed before all others, since when exploring $t$ in a breadth-first manner, the nodes on a given level should be enumerated from left to right. The function `list_nodes_depth_first()` is modified into the function `list_nodes_breadth_first()` in the next cell, in which the function is then called with `t` passed as argument. The cell is followed with a cell that traces execution of that function call, replacing the forests stored in the queue with the roots of their trees:

In [ ]:
def list_nodes_breadth_first(t):
    root = next(iter(t))
    roots_and_forests = deque([(root, t[root])])
    while roots_and_forests:
        root, forest = roots_and_forests.pop()
        print(root, end=' ')
        if forest:
            roots_and_forests.extendleft(forest.items())

list_nodes_breadth_first(t)

In [ ]:
root = next(iter(t))
roots_and_forests = deque([(root, t[root])])
print('Queue now (with trees changed to roots):\n     ',
      [(root, [k for k in t[root]])]
     )
while roots_and_forests:
    root, forest = roots_and_forests.pop()
    print()
    if forest:
        print('Node output:', root, '\t\tRoots of forest to process:',
              [k for k in forest]
             )
    else:
        print('Node output:', root, '\t\tEmpty forest')
    if forest:
        roots_and_forests.extendleft(forest.items())
    print('Queue now (with trees changed to roots):\n     ',
          [(root, [k for k in forest] if forest else None)
                 for (root, forest) in roots_and_forests
          ]
         )

`list_paths_depth_first()` is modified into `list_paths_breadth_first()` as follows:

In [ ]:
def list_paths_breadth_first(t):
    root = next(iter(t))
    paths_and_forests = deque([([root], t[root])])
    while paths_and_forests:
        path, forest = paths_and_forests.pop()
        print(path)
        if forest:
            paths_and_forests.extendleft((path + [k], forest[k])
                                              for k in forest
                                        )
        
list_paths_breadth_first(t)

Getting back to both trees for the goal grandparent(john, X):

* Exploring the first tree in a depth-first manner yields both solutions fastest, after which the rest of the exploration is "for nothing".
* Exploring the second tree in a depth-first manner yields both solutions at the very end.
* Exploring the first and second trees in a breadth-first manner yields both solutions at the very end.

Getting back to both trees for the goal join(X, X, Y):

* Exploring the first tree in a depth-first or breadth-first manner makes no difference; the solutions are generated one by one, and the exploration would have to be interrupted at some point as it could go on forever and produce infinitely many solutions.
* Exploring the second tree in a breadth-first manner is hardly different to exploring the first tree in a breadth-first manner, one node being visited before rather than after the production of a given solution.
* Exploring the second tree in a depth-first manner traps the search in an infinite descent along the leftmost branch, with no solution being ever produced.

Solving goals relative to a given definite logic program by a breadth-first exploration of the associated search tree is a __complete__ proof procedure: every solution is eventually produced. This is an immediate consequence of the fact that such a tree is __finitely branching__. On the other hand, as we have just observed, a depth-first exploration does not offer a complete proof procedure. Usually, Prolog's proof engine implements a depth-first search, hence an incomplete proof procedure: it expects users to properly order the rules that make up the program, and to properly order in a rule the atoms that make up its body, so that the search trees associated with goals of interest have "a good shape" and make depth-first search a most efficient procedure. We will write a Prolog interpreter where we can chose to explore a search tree either in a depth-first or in a breadth-first manner (the difference is minor and essentially relies on using either a stack or a queue in pretty much the same way, as we have previously observed). The exploration of a search tree is the last part of the work the interpreter has to do. First, we need to be able to parse the rules that make up a definite logic program, as well as perform a number of operations such as separate head and body from a rule, identify which variables occur in an atom, perform substitution of variables by terms in an atom, etc. We first define a couple of helper functions to consistently extend a dictionary with a new key-value pair, or all the key-value pairs from another dictionary:

In [ ]:
def consistently_add_to(data_pair, mapping):
    return mapping.setdefault(data_pair[0], data_pair[1]) == data_pair[1]

mapping = {'A': 1, 'B': 2, 'C': 3}
consistently_add_to(('B', 4), mapping), mapping
consistently_add_to(('B', 2), mapping), mapping
consistently_add_to(('D', 4), mapping), mapping

In [ ]:
def consistently_merge_to(mapping_1, mapping_2):
    for data_pair in mapping_1.items():
        if not consistently_add_to(data_pair, mapping_2):
            return False
    return True

mapping = {'A': 1, 'B': 2, 'C': 3}
consistently_merge_to({'B': 2, 'C': 4, 'D': 5}, mapping), mapping
consistently_merge_to({'B': 2, 'C': 3}, mapping), mapping
consistently_merge_to({'C': 3, 'D': 4, 'E': 5}, mapping), mapping

We define three classes, `Expression`, `Term` and `Atom`, with `Term` and `Atom` inheriting from `Expression`, considering that terms and atoms are two kinds of expressions.

Generalising on the examples previously examined:

* an atom is built from an $n$-ary predicate symbol ($n\in\mathbf N$) and $n$ terms;
* a term is a variable or is built from an $n$-ary function symbol ($n\in\mathbf N$) and $n$ terms.

So the outermost symbol in an atom is a predicate symbol while all other symbols are function symbols or variables; all symbols in a term are function symbols or variables. Prolog function symbols, predicate symbols and variables are built from alphanumeric characters and underscores, with function and predicate symbols starting with lowercase letters, and variables starting with uppercase letters or underscores.

Trees are most appropriate to represent expressions. To give `Expression` objects the structure of trees that represent expressions, we use two attributes, `root` and `children`. Given an object $O$ of classs `Expression` meant to represent an expression $E$:

* the value of `root` for $O$ will be set to the string that denotes $E$'s outermost symbol $s$ (a predicate symbol if $E$ is an atom, a function symbol if $E$ is a term);
* if $n$ is $s$'s arity, the value of `children` for $O$ will be set to a list of length $n$ consisting of $n$ `Expression` objects, that represent the $n$ terms that are the arguments to $s$ in $E$, listed from left to right (so the list is empty in case $E$ is a nullary predicate symbol, a constant, or a variable).

`Expression` objects will be created by parsing strings, giving atoms and terms a tree structure. We implement `__str__()` in `Expression` for what can be seen as the reverse operation: get the (properly formatted) string from the representing tree. When testing `__str__()` below, we define `Expression` objects "by hand"; parsing methods will be defined next:

In [ ]:
class Expression:
    def __init__(self, root, children=None):
        self.root = root
        if children is None:
            children = []
        self.children = children

    def __str__(self):
        return self.root if not self.children\
                    else ''.join((self.root, '(',
                                 ', '.join(child.__str__()
                                                for child in self.children
                                          ), ')'
                                 )
                                )


class Term(Expression):
    class TermError(Exception):
        pass

    def __init__(self, root, children=None):
        super().__init__(root, children)  

        
class Atom(Expression):
    class AtomError(Exception):
        pass

    def __init__(self, root, children=None):
        super().__init__(root, children)

In [ ]:
# bob: constant
print(Term('bob'))
# l: binary function symbol
# H, T: variables
print(Term('l', [Term('H'), Term('T')]))
# l: binary function symbol
# e: constant
# H, T: variables
print(Term('l', [Term('e'), Term('l', [Term('H'), Term('T')])]))

print()

# on: nullary predicate symbol
print(Atom('on'))
# happy: unary predicate symbol
# john: constant
print(Atom('happy', [Term('john')]))
# mother: binary predicate symbol
# jane, sandra: constants
print(Atom('mother', [Term('jane'), Term('sandra')]))
# join: binary predicate symbol
# l: binary function symbol
# H, T, X, Y: variables
print(Atom('join', [Term('l', [Term('H'), Term('T')]), Term('X'),
                    Term('l', [Term('H'), Term('Y')])
                   ]
          )
     )

To parse an atom or a term represented as a string, we will first get rid of all spaces in the string, if any, and convert the resulting string to a list $L$ of characters, from last character in the string to first character in the string, so that characters can be efficiently consumed by popping them off the end of the list, as opposed to removing them from the beginning of the list. Opening and closing parentheses and commas will need special processing. The rest is predicate and function symbols and variables, which will be dealt with thanks to the function `parse_word()` of the `Expression` class. This function receives as argument what remains of $L$, assumed to be at the stage where a predicate or function symbol or a variable is to be parsed; so $L$ then ends in the characters that make up that predicate or function symbol or variable in reverse order; `parse_word()` consumes those characters from $L$ and returns the predicate or function symbol or the variable as a string:

In [ ]:
class Expression(Expression):
    def parse_word(characters):
        word = [characters.pop()]
        while characters and (characters[-1].isalnum()
                              or characters[-1] == '_'
                             ):
            word.append(characters.pop())
        return ''.join(word)

In [ ]:
characters = list(reversed('bob'.replace(' ', '')))
characters
Expression.parse_word(characters)
characters

print()

characters = list(reversed('happy(  john  )'.replace(' ', '')))
characters
Expression.parse_word(characters)
characters

print()

# Could be what remains to be parsed in "join(l(H, T), X, l(H, Y))"
# after "join(l(H, T), X, l(" has been parsed already.
characters = list(reversed('H,  Y))'.replace(' ', '')))
characters
Expression.parse_word(characters)
characters

Let an atom or a term $E$ be given and let $L$ be the list of all nonspace characters in $E$ in reverse order. We will define in `Expression` two functions, `parse_subitem()` and `parse_subitem_sequence()`, meant to operate in a way that we now describe. To parse $E$ (possibly as a subexpression of a larger expression), the function `parse_subitem()` will be called with $L$ passed as argument. It will first check that $E$ indeed starts with (that is, $L$ indeed ends in) a character that can be the beginning of a predicate or function symbol or a variable and call `parse_word()`, passing $L$ as argument. If $E$ is a nullary predicate or function symbol or a variable, that predicate or function symbol or variable will be returned by `parse_word()` and $L$ will have become empty. Otherwise, $E$ is of the form $\sigma(t_1,\dots,t_n)$ for some nonzero $n\in\mathbf N$, predicate or function symbol $\sigma$, and terms $t_1$, ..., $t_n$. Then `parse_word()` will return $\sigma$, having consumed all characters that make up $\sigma$; `parse_subitem()`, finding out that $L$ is not empty, will then check that $L$ indeeds ends in `(`, consume that character (that it, pop `(` off the end of $L$), and call `parse_subitem_sequence()`, passing $L$ as argument, whose purpose is to parse $t_1,\dots, t_n$ and return a list with as members, the $n$ `Term` objects $o_1$, ..., $o_n$ that represent $t_1$, ..., $t_n$. At this stage, all characters occurring in $t_1$, ..., $t_n$ and the separating commas will have been consumed, and the only task left for `parse_subitem()` to complete will be to check that `)` is now the last symbol in $L$ (it should also be the only symbol in $L$ in case $E$ is the whole expression to parse, but that will not be up to `parse_subitem()` to check: `parse_subitem()` does not know whether it parses a whole expression or a subexpression of a larger expression), consume it, and create an `Atom` or a `Term` object whose `root` attribute should be set to $\sigma$ and whose `children` attribute should be set to the list $[o_1,\dots,o_n]$. Recall that `parse_subitem_sequence()` will have to parse $t_1,\dots, t_n$. It will do so with a first call to `parse_subitem()` to create a `Term` object $o_1$ that represents $t_1$. It will then find out that $L$ now ends in a comma, consume it, and make a second call to `parse_subitem()` to create a `Term` object $o_2$ that represents $t_2$. Eventually, it will make a last call to `parse_subitem()` to create a `Term` object $o_n$ that represents $t_n$, find out that $L$ does not end in a comma, assume that the whole sequence has been successfully parsed, and return $[o_1,\dots,o_n]$ to its caller (the original `parse_subitem()` call).

Before we implement `parse_subitem()` and `parse_subitem_sequence()`, we define skeleton functions to illustrate how `parse_subitem()` and `parse_subitem_sequence()` are meant to call each other, and how characters are consumed:

In [ ]:
def parse_subitem_skeleton(characters, depth=0):
    print('  ' * depth, 'Start parsing subitem, characters left:',
          ''.join(reversed(characters))
         )
    Expression.parse_word(characters)
    if not characters or characters[-1] != '(':
        print('  ' * depth, 'End parsing subitem, characters left:',
              ''.join(reversed(characters))
             )
        return
    # Popping (
    characters.pop()
    parse_subitem_sequence_skeleton(characters, depth + 1)
    # Popping )
    characters.pop()
    print('  ' * depth, 'End parsing subitem, characters left:',
          ''.join(reversed(characters))
         )

def parse_subitem_sequence_skeleton(characters, depth):
    print('  ' * depth, 'Start parsing subitem sequence, characters left:',
          ''.join(reversed(characters))
         )
    expressions = []
    while True:
        expressions.append(parse_subitem_skeleton(characters, depth + 1))
        if characters[-1] != ',':
            print('  ' * depth,
                  'End parsing subitem sequence, characters left:',
                  ''.join(reversed(characters))
                 )
            return
        # Popping ,
        characters.pop()
        
parse_subitem_skeleton(list(reversed('bob')))
print()
parse_subitem_skeleton(list(reversed('happy(john)')))
print()
parse_subitem_skeleton(list(reversed('l(e,l(H,T))')))

Now for the implementation of `parse_subitem()` and `parse_subitem_sequence()`. Besides the list of characters $L$, both have a second parameter, `item_type`, meant to be set to a string that represents the type of expression to parse, namely, either `Atom` or `Term`. At this stage of the discussion, `parse_subitem()` seems to be useful to parse either atoms or terms, whereas `parse_subitem_sequence()` seems to be useful only to parse sequences of terms. But to parse the bodies of the rules of a logic program, we will have to parse sequence of atoms; `parse_subitem_sequence()` will be perfectly suitable for the task, with the second parameter of `parse_subitem_sequence()` set to `'Atom'`. Still, as `parse_subitem_sequence()` will be more often used to parse sequences of terms, we give its parameter `item_type` the default value of `'Term'`, which allows `parse_subitem()` to call `parse_subitem_sequence()` with no other argument but $L$.

It was tacitly assumed that `parse_subitem_skeleton()` and `parse_subitem_sequence_skeleton()` would be used only to parse a syntactically correct expression; `parse_subitem()` and `parse_subitem_sequence()` make no such assumption, so also check for syntactic correctness, and return `None` whenever they find out that there are characters that cannot be for a syntactically correct expression:

* Before it calls `parse_word()`, `parse_subitem()` checks that there is indeed at least one character to parse, and that:
  * in case the expression being parsed is an atom, `parse_word()` should return a predicate symbol, hence the first character to parse should be a lowercase letter;
  * in case the expression being parsed is a term, `parse_word()` should return a function symbol or a variable, hence the first character to parse should be a letter or an underscore (which is equivalent to `isidentifier()` identifying the string consisting of that single character as an identifier).
* In case `parse_subitem()` does not process a nullary predicate or function symbol or a variable, because it finds an opening parenthesis and calls `parse_subitem_sequence()`, and provided that the latter successfully parses a sequence of expressions and does not return `None` (otherwise, `parse_subitem()` should itself return `None`), it needs to finally check that there are some characters left, with as first character to parse, a closing parenthesis.
* As for `parse_subitem_sequence()`, provided that all calls to `parse_subitem()` successfully parse an expression and do not return `None` (otherwise, `parse_subitem_sequence()` should itself return `None`), it has to verify that there is at least one character left before checking whether that character is a comma, indicating that furher expressions still have to be parsed in the sequence being currently dealt with; if that is not the case, `parse_subitem_sequence()` does not have to return `None`: `parse_subitem()` will, as caller to `parse_subitem_sequence()`, return `None`.

`parse_subitem()` creates `Atom` or `Term` objects (the objects in the sequence of objects created by `parse_subitem_sequence()` are created by `parse_subitem()`); the `item_type` parameter informs `parse_subitem()` of the appropriate type. The `globals()` function returns a dictionary with `'Atom'` and `'Term'` as particular keys, with as corresponding values, the types themselves.

`parse_subitem()` and `parse_subitem_sequence()` are both meant to be used as helper functions. We extend the `Expression` class with a function, `parse_item()`, that:

* gets from a string $\sigma$ a list of characters as previously described (nonspace characters in $\sigma$ removed, and listed from last to first);
* calls by default `parse_subitem()`, because the `parse_single_subitem` parameter of `parse_item()` evaluates to `True`, the default value, which is suitable for the case where $\sigma$ represents an atom or a term, but as pointed out above, there will be a need to parse rule bodies, that is, sequences of atoms, and then `parse_single_subitem` will be set to `False` to let `parse_item()` call `parse_subitem_sequence()` instead of `parse_subitem()`;
* passes as argument to `parse_subitem()` or `parse_subitem_sequence()` the type of object or objects to be created, thanks to the `item_type` parameter of `parse_item()`, set to `Atom` by default;
* is meant to fully parse the string provided as first argument to `parse_item()`, because that argument is not meant to be a subexpression of a larger expression, hence returns `None` in case some characters still remain after `parse_subitem()` or `parse_subitem_sequence()` have returned an object or a sequence of objects (also returning `None` otherwise):

In [ ]:
class Expression(Expression):
    def parse_subitem(characters, item_type):
        if not characters or not {'Atom': lambda c: c.islower(),
                                  'Term': lambda c: c.isidentifier()
                                 }[item_type](characters[-1]):
            return
        symbol = Expression.parse_word(characters)
        if not characters or characters[-1] != '(':
            return globals()[item_type](symbol)
        characters.pop()
        subitems = Expression.parse_subitem_sequence(characters)
        if not subitems or not characters or characters.pop() != ')':
            return
        return globals()[item_type](symbol, subitems)

    def parse_subitem_sequence(characters, item_type='Term'):
        expressions = []
        while True:
            expression = Expression.parse_subitem(characters, item_type)
            if not expression:
                return
            expressions.append(expression)
            if not characters or characters[-1] != ',':
                return expressions
            characters.pop()
            
    def parse_item(expression, item_type='Atom', parse_single_subitem=True):
        characters = list(reversed(expression.replace(' ', '')))
        item = Expression.parse_subitem(characters, item_type)\
                  if parse_single_subitem\
                  else Expression.parse_subitem_sequence(characters, item_type)
        if not item or characters:
            return
        return item

In [ ]:
expression = Expression.parse_item('bob', 'Term')
print(type(expression), expression)

expression = Expression.parse_item('l(e,l(H,T))', 'Term')
print(type(expression), expression)

expression = Expression.parse_item('father(bob,sandra)')
print(type(expression), expression)

expression = Expression.parse_item('join(l(H,T),X,l(H,Y))')
print(type(expression), expression)

An atom or term can be successfully parsed while still being an invalid Prolog expression because a predicate or function symbol is used with different arities, or because a symbol is used both as predicate and function symbols. We define in `Expression` a function, `collected_symbols()`, meant to collect the symbols (not variables) that occur in an `Expression` object, in the form of a dictionary, with symbols as keys and symbols' arities as values. We let `collected_symbols()` ignore the outermost symbol of the expression by letting its second parameter, `include_root`, take the default value of `False`. That is suitable for `Expression` objects that are more precisely of type `Atom` since the outermost symbol (the value of their `root` attribute) is a predicate symbol, to be distinguished from the other symbols, all function symbols (besides variables), that occur in the expression that the object represents. On the other hand, for `Expression` objects that are more precisely of type `Term`, setting `include_root` to `True` is appropriate to collect all function symbols, including the outermost one. The lambda expression `top_symbol`, defined in `Expression` and called by `collected_symbols()`, is meant to return the outermost symbol and the arity of an `Expression` object that does not represent a variable. The core of the work is performed recursively thanks to the helper functions `consistently_add_to()` and `consistently_merge_to()`.

We extend the `Atom` class with a method `predicate_and_function_symbols()` that collects all function symbols in an object of type `Atom` (provided they are consistently used), and separately, the predicate (outermost) symbol (provided it is different to the function symbols); essentially, the method calls `collected_symbols()`.

We extend the `Term` and `Atom` classes with two functions, `parse_term()` and `parse_atom()`, respectively, that essentially call the `parse_item()` function of the `Expression` class, informing it of the kind of expression, hence the type of object, to be created (explicitly for terms, implicitly for atoms). The functions call `collected_symbols()`, with `parse_atom()` performing some extra work, to check that the parsed expression is a correct Prolog term or atom, respectively: 

In [ ]:
class Expression(Expression):
    top_symbol = lambda tree: (tree.root, len(tree.children))\
                              if tree.root[0].islower() else None

    def collected_symbols(self, include_root=False):
        symbols = {}
        if include_root:
            root = Expression.top_symbol(self)
            if root:
                symbols[root[0]] = root[1]
        for child in self.children:
            symbol = Expression.top_symbol(child)
            if symbol\
               and not consistently_add_to(symbol, symbols)\
               and not consistently_merge_to(child.collected_symbols(symbol),
                                             symbols
                                            ):
                return
        return symbols


class Term(Term, Expression):
    def parse_term(expression):
        term = Expression.parse_item(expression, 'Term')
        if not term:
            raise Term.TermError(f'{expression}: syntactically incorrect')
        if term.collected_symbols(True) is None:
            raise Term.TermError('Same function symbol used with '
                                 f'different arities in {expression}'
                                )
        return term


class Atom(Atom, Expression):
    def predicate_and_function_symbols(self):
        return (self.root, len(self.children)), self.collected_symbols()
    
    def parse_atom(expression):
        atom = Expression.parse_item(expression)
        if not atom:
            raise Atom.AtomError(f'{expression}: syntactically incorrect')
        function_symbols = atom.collected_symbols()
        if function_symbols is None:
            raise Atom.AtomError('Same function symbol used with '
                                 f'different arities in {expression}'
                                )
        if atom.root in function_symbols:
            raise Atom.AtomError('Predicate symbol one of function symbols')
        return atom

In [ ]:
term = Term.parse_term('X')
print(term)
term.collected_symbols(True)

term = Term.parse_term('bob')
print(term)
term.collected_symbols(True)

term = Term.parse_term('l(e, l(H, T))')
print(term)
term.collected_symbols(True)

atom = Atom.parse_atom('father(bob, sandra)')
print(atom)
atom.predicate_and_function_symbols()

atom = Atom.parse_atom('join(l(H, T), X, l(H, Y))')
print(atom)
atom.predicate_and_function_symbols()

Having parsed atoms and terms in the form of `Atom` and `Term` objects, respectively, we can now define a number of functions and methods to operate on `Expression` objects as needed by a Prolog interpreter. It is necessary to be able and find out whether an expression is a variable, and collect all variables that occur in an expression:

In [ ]:
class Expression(Expression):
    def is_variable(self):
        return self.root[0].isupper() or self.root[0] == '_' 

    def variables(self):
        variables = {self.root} if self.is_variable() else set()
        for child in self.children:
            variables.update(child.variables())
        return variables


class Term(Term, Expression):
    pass


class Atom(Atom, Expression):
    pass

In [ ]:
Term.parse_term('x').variables()
Term.parse_term('X').variables()
Term.parse_term('f(X, a, X)').variables()
Term.parse_term('f(X_0, Y, X_0)').variables() == {'Y', 'X_0'}
Term.parse_term('f(c, f(X, f(a, Z, b), '
                'f(f(X, Z, U), a, T)), f(a, U, a))'
               ).variables() == {'U', 'T', 'X', 'Z'}

Prolog considers underscores within a rule as independent variables. For instance, the fact $\mathit{loves}(\_,\_).$ is meant to express that everyone loves everyone (including oneself), not only that everyone loves oneself: the two occurrences of $\_$ denote arbitrary, independant individuals, and $\mathit{loves}(\_,\_)$ has the same intended meaning as $\mathit{loves}(\_0,\_1)$. A Prolog interpreter needs to treat all occurrences of a given variable in a rule as denotations of the same individual (as a consequence, a Prolog interpreter needs to make sure that whenever an occurrence of a variable in a rule is replaced by a term, then all other occurrences of the variable in the rule are replaced by the term). To that aim, we define in `Expression` a recursive method, `individualise_underscores()`, meant to let in an expression each occurrence of an underscore, used as a full name for a variable, be followed by a unique natural number. We first explain how the method operates thanks to a tracing function:

In [ ]:
def trace_individualise_underscores(expression, variables, index, depth):
    print('  ' * depth, f'Received index is {index}, processing', expression)
    if expression.root == '_':
        while True:
            index += 1
            next_underscore = '_' + str(index)
            if next_underscore not in variables:
                expression.root = next_underscore
                print('  ' * depth, f'_ changed to {expression}, returned '
                      'index is', index
                     )
                return index
    if not expression.children:
        print('  ' * depth, 'Returned index is', index)
        return index
    for child in expression.children:
        index = trace_individualise_underscores(child, variables, index,
                                                depth + 1
                                               )
    print('  ' * depth, 'Returned index is', index)
    return index
    
atom = Atom.parse_atom('p(g(h(a, _), X), _2, h(a, _), _)')
variables = atom.variables()
print('Variables in expression are:', variables, end='\n\n')
trace_individualise_underscores(atom, variables, -1, 0)
print('expression now is:', atom)

The implementation of `individualise_underscores()` in the following cell is a straightforward adaptation of `trace_individualise_underscores()`. When dealing with a fact, `individualise_underscores()` can be called on the `Atom` object that captures the fact, and the default arguments are appropriate. When dealing with a more general rule, `individualise_underscores()` can be called on the rule's head, but the argument `variables` should be given as value the set $S$ of variables that occur in the whole rule, not just in the rule's head; that call would return an integer $i_1$. Then `individualise_underscores()` can be called on the first atom in the rule's body, with `variables` still set to $S$, but with the argument `index` given the value $i_1$; that call would return an integer $i_2$. Then `individualise_underscores()` can be called on the second atom in the rule's body, if any, with `variables` still set to $S$, but with the argument `index` given the value $i_2$... That is a way to give each underscore that occurs in the rule a unique name, not occurring anywhere in the rule. That is not something to see in action yet as for now, we are only dealing with expressions, not rules and logic programs:

In [ ]:
class Expression(Expression):
    def individualise_underscores(self, variables=None, index=-1):
        if variables is None:
            variables = self.variables()
        if self.root == '_':
            while True:
                index += 1
                next_underscore = '_' + str(index)
                if next_underscore not in variables:
                    self.root = next_underscore
                    return index
        if not self.children:
            return index
        for child in self.children:
            index = child.individualise_underscores(variables, index)
        return index


class Term(Term, Expression):
    pass


class Atom(Atom, Expression):
    pass

In [ ]:
atom = Atom.parse_atom('p(g(h(a, _), X), _2, h(a, _), _)')
atom.individualise_underscores()
print(atom)

A Prolog interpreter needs to be able to substitute in an expression $E$ all occurrences of some of the variables that occur in $E$ by terms; sometimes, the terms will simply be computed fresh variables, for a particular kind of substitution referred to as a _renaming of variables in $E$_. We extend `Expression` with a function, `fresh_variables()`, meant to take two arguments, `variables` and `reserved_variables`, both expected to be sets of variables, and return a dictionary $D$ whose keys are the members of both `variables` and `reserved_variables`, and whose values are pairwise distinct variables, different to those in both sets. The intention is that given an expression $E$, `variables` will denote the set of variables that occur in $E$; any such variable $v$ that happens to belong to `reserved_variables` can then be replaced by $D[v]$ in $E$, resulting in an expression where fresh variables have replaced those in both `variables` and `reserved_variables`. We let the name of a variable that belongs to both `variables` and `reserved_variables` be followed by an underscore and the smallest possible natural number for the mapped to fresh variable:

In [ ]:
class Expression(Expression):
    def fresh_variables(variables, reserved_variables):
        substitutions = {}
        # Any variable Var that occurs in both variables and
        # reserved_variables will be renamed to Var_i where i is the
        # least natural number that makes Var_i a new variable (that is,
        # occurring neither in variables nor in reserved_variables nor
        # in the set of variables that have been created so far, if
        # any).
        for variable in variables & reserved_variables:
            i = 0
            while ''.join((variable, '_', str(i))) in variables\
                                                      | reserved_variables:
                i += 1
            substitutions[variable] = ''.join((variable, '_', str(i)))
        return substitutions

In [ ]:
Expression.fresh_variables({'a'}, set())
Expression.fresh_variables({'Y'}, {'X'})
Expression.fresh_variables({'X'}, {'X'})
Expression.fresh_variables({'Y', 'Z'}, {'X', 'Y'})
Expression.fresh_variables({'U', 'Y', 'Z'}, {'X', 'Y', 'Z'}) ==\
                                           {'Z': 'Z_0', 'Y': 'Y_0'}

When the Prolog interpreter needs to rename variables in an expression $E$, it  actually always has to leave $E$ untouched and perform the renaming on a copy of $E$. The `deepcopy` function from the `copy` module offers a good solution to cloning an `Expression` object. Compare:

In [ ]:
L = [0]
# Alternatively, use the copy method of the List class:
# L_copy = L.copy()
L_copy = copy(L)
L_copy[0] = 1
L_copy, L

L = [[0]]
# Alternatively, use the copy method of the List class:
# L_copy = L.copy()
L_copy = copy(L)
L_copy[0][0] = 1
L_copy, L

L = [[0]]
L_deepcopy = deepcopy(L)
L_deepcopy[0][0] = 1
L_deepcopy, L

We extend `Expression` with a method, `rename_variables()`, that recursively renames variables in a copy of an expression, the renaming taking the form of a dictionary mapping variables to replace to replacing variables:

In [ ]:
class Expression(Expression):
    def rename_variables(self, substitution):
        return deepcopy(self)._rename_variables(substitution)

    def _rename_variables(self, substitution):
        if self.is_variable():
            if self.root in substitution:
                self.root = substitution[self.root]
        else:
            for child in self.children:
                child._rename_variables(substitution)
        return self


class Term(Term, Expression):
    pass


class Atom(Atom, Expression):
    pass

In [ ]:
atom = Atom.parse_atom('join(l(H, T),X, l(H, Y))')
print(atom.rename_variables({'X': 'A', 'Y': 'B', 'Z': 'C'}))
print(atom)

More general substitutions of variables by terms in an expression $E$ will have to be performed sometimes in $E$ itself, sometimes in a copy of $E$. We extend `Expression` with two methods, `substitute()` and `substitute_in_copy()`, for both kinds of substitutions, the latter just calling the former on a copy of the object it applies the method to. When a term $t$ replaces a variable $v$ in $E$, $t$ might itself contain variables that belong to the domain of the substitution. In that case, we apply the substitution again, many times if needed. If the substitution requested to replace a variable by itself, or requested to replace a variable $v_1$ by a variable $v_2$ and the other way around, the procedure would loop forever. The kind of substitution that will be used in practice will not make this possible: after a finite number of applications of the substitution, the resulting expression will contain no occurrence of a variable in the domain of the substitution. Note that the assignment in the body of `substitute()` cannot be replaced by `self = substitution[self.root]`:

In [ ]:
class Expression(Expression):
    def substitute_in_copy(self, substitution):
        return deepcopy(self).substitute(substitution)

    def substitute(self, substitution):
        if self.is_variable():
            if self.root in substitution:
                self.root, self.children = substitution[self.root].root,\
                                           substitution[self.root].children
                self.substitute(substitution)
        else:
            for child in self.children:
                child.substitute(substitution)
        return self


class Term(Term, Expression):
    pass


class Atom(Atom, Expression):
    pass

In [ ]:
term = Term.parse_term('f(a, X, g(Y, b), h(h(h(Z))))')
print(term.substitute({'X': Term.parse_term('X_0'), 'Z': Term.parse_term('Z_0')
                      }
                     )
     )

term = Term.parse_term('h(h(f(U, Y, g(Z, Z))))')
print(term.substitute({'Z': Term.parse_term('V')}))

term = Term.parse_term('f(a, X, g(Y, b), h(h(h(Z))))')
print(term.substitute({'X_0' : Term.parse_term('h(Z_0)'),
                       'X': Term.parse_term('X_0'), 'Z_0': Term.parse_term('c')
                      }
                     )
     )

At the heart of Prolog lies the _unification algorithm_. It computes a most general unifier (_mgu_) for two expressions $E_1$ and $E_2$, that is, a substitution $\theta$ such that:

* applying $\theta$ to $E_1$ and $E_2$ results in the same expression;
* for any substitution $\psi$ that applied to $E_1$ and $E_2$, results in the same expression, there exists a substitution $\nu$ such that applying $\psi$ to $E_1$ and $E_2$ is the same as applying $\theta$ to $E_1$ and $E_2$, and then $\nu$ to the resulting expressions.

For instance, if $E_1$ is $f(X_1, h(X_1), X_2)$ and $E_2$ is $f(g(X_3), X_4, X_3)$, then the substitution that maps $X_1$ to $g(X_3)$, $X_2$ to $X_3$ and $X_4$ to $h(g(X_3))$ is an mgu for $E_1$ and $E_2$; when applied to $E_1$ and $E_2$, it results in the expression $f(g(X_3), h(g(X_3)), h(g(X_3)))$. If $a$ is a constant, then the substitution that maps $X_1$ to $g(a)$, $X_2$ to $a$ and $X_4$ to $h(g(a))$ results in the same expression when applied to $E_1$ and $E_2$, namely, $f(g(a), h(g(a)), h(g(a)))$; but it suffices to apply the substitution that maps $X_3$ to $a$ to get $f(g(a), h(g(a)), h(g(a)))$ from $f(g(X_3), h(g(X_3)), h(g(X_3)))$.

So a most general unifier for two expressions $E_1$ and $E_2$ is a substitution $\theta$ that specialises $E_1$ and $E_2$ to the same expression in the most general way: any other substitution that specialises $E_1$ and $E_2$ to the same expression can be obtained by instantiating $\theta$. A most general unifier is unique up to a renaming of variables.

The following tracing function, `trace_unify_identities()`, is meant to explain and illustrate the unification algorithm. It takes as argument a list of pairs of expressions. To compute an mgu for two expressions $E_1$ and $E_2$, `trace_unify_identities()` is called with $[(E_1,E_2)]$ provided as argument:

In [ ]:
def trace_unify_identities(identities):
    mgu = {}
    while identities:
        print('Identities left to process: ', end='')
        print(', '.join(f'{identity[0]} = {identity[1]}'
                            for identity in identities
                       )
             )
        expression_1, expression_2 = identities.pop()
        print('    Dealing with:', expression_1, '=', expression_2)
        if expression_1.root == expression_2.root:
            for (expr_1, expr_2) in zip(expression_1.children,
                                        expression_2.children
                                       ):
                print('    Adding:', expr_1, '=', expr_2)
                identities.append([expr_1, expr_2])
        elif expression_1.is_variable():
            # Occurrence check (omitted in most Prolog implementations)
            if expression_1.root in expression_2.variables():
                print(f'    {expression_1.root} occurs in {expression_2},',
                      'giving up!'
                     )
                return
            print('    Extending mgu with:', expression_1.root, '->',
                  expression_2
                 )
            mgu[expression_1.root] = expression_2
            for identity in identities:
                print('    Changing (or not)', identity[0], '=', identity[1],
                      'to ', end=''
                     )
                for i in range(2):
                    identity[i] = identity[i].substitute_in_copy(
                                            {expression_1.root: expression_2}
                                                                )
                print(identity[0], '=', identity[1])
        elif expression_2.is_variable():
            print('    Adding', expression_2, '=', expression_1)
            identities.append([expression_2, expression_1])
        else:
            print('    Equality cannot be satisfied, giving up!')
            return
        print()
    print('Mgu:')
    print('\n'.join(f'    {var} -> {mgu[var]}' for var in mgu))
    return mgu

In [ ]:
term_1 = Term.parse_term('X')
term_2 = Term.parse_term('X')
mgu = trace_unify_identities([(term_1, term_2)])

if mgu is not None:
    print('\nExpressions after application of mgu:')
    print('   ', term_1.substitute(mgu))
    print('   ', term_2.substitute(mgu))

In [ ]:
term_1 = Term.parse_term('X')
term_2 = Term.parse_term('a')
mgu = trace_unify_identities([(term_1, term_2)])

if mgu is not None:
    print('\nExpressions after application of mgu:')
    print('   ', term_1.substitute(mgu))
    print('   ', term_2.substitute(mgu))

In [ ]:
term_1 = Term.parse_term('X')
term_2 = Term.parse_term('Y')
mgu = trace_unify_identities([(term_1, term_2)])

if mgu is not None:
    print('\nExpressions after application of mgu:')
    print('   ', term_1.substitute(mgu))
    print('   ', term_2.substitute(mgu))

In [ ]:
term_1 = Term.parse_term('f(X, Y)')
term_2 = Term.parse_term('f(Y, X)')
mgu = trace_unify_identities([(term_1, term_2)])

if mgu is not None:
    print('\nExpressions after application of mgu:')
    print('   ', term_1.substitute(mgu))
    print('   ', term_2.substitute(mgu))

In [ ]:
term_1 = Term.parse_term('f(X1, h(X1), X2)')
term_2 = Term.parse_term('f(g(X3), X4, X3)')
mgu = trace_unify_identities([(term_1, term_2)])

if mgu is not None:
    print('\nExpressions after application of mgu:')
    print('   ', term_1.substitute(mgu))
    print('   ', term_2.substitute(mgu))

In [ ]:
term_1 = Term.parse_term('f(X1 ,g(X2, X3), X2, b)')
term_2 = Term.parse_term('f(g(h(a, X5), X2), X1, h(a, X4), X4)')
mgu = trace_unify_identities([(term_1, term_2)])

if mgu is not None:
    print('\nExpressions after application of mgu:')
    print('   ', term_1.substitute(mgu))
    print('   ', term_2.substitute(mgu))

In [ ]:
term_1 = Term.parse_term('f(X, a)')
term_2 = Term.parse_term('f(b, X)')
mgu = trace_unify_identities([(term_1, term_2)])

if mgu is not None:
    print('\nExpressions after application of mgu:')
    print('   ', term_1.substitute(mgu))
    print('   ', term_2.substitute(mgu))

In [ ]:
term_1 = Term.parse_term('f(X, Y, U)')
term_2 = Term.parse_term('f(Y, U, g(X))')
mgu = trace_unify_identities([(term_1, term_2)])

if mgu is not None:
    print('\nExpressions after application of mgu:')
    print('   ', term_1.substitute(mgu))
    print('   ', term_2.substitute(mgu))

The implementation of the unification algorithm in `Expression` should now be clear:

In [ ]:
class Expression(Expression):
    def unify_identities(identities):
        mgu = {}
        while identities:
            expression_1, expression_2 = identities.pop()
            if expression_1.root == expression_2.root:
                identities.extend([expr_1, expr_2] for (expr_1, expr_2) in
                                                     zip(expression_1.children,
                                                         expression_2.children
                                                        )
                                 )
            elif expression_1.is_variable():
                # Occurrence check (omitted in most Prolog
                # implementations)
                if expression_1.root in expression_2.variables():
                    return
                mgu[expression_1.root] = expression_2
                for identity in identities:
                    for i in range(2):
                        identity[i] = identity[i].substitute_in_copy(
                                              {expression_1.root: expression_2}
                                                                    )
            elif expression_2.is_variable():
                identities.append([expression_2, expression_1])
            else:
                return
        return mgu
            
    def unify(self, expression):
        return Expression.unify_identities([[self, expression]])


class Term(Term, Expression):
    pass


class Atom(Atom, Expression):
    pass

In [ ]:
term = Term.parse_term('X')
mgu = term.unify(Term.parse_term('X'))
{var: str(mgu[var]) for var in mgu}

term = Term.parse_term('X')
mgu = term.unify(Term.parse_term('a'))
{var: str(mgu[var]) for var in mgu}

term = Term.parse_term('X')
mgu = term.unify(Term.parse_term('Y'))
{var: str(mgu[var]) for var in mgu}

term = Term.parse_term('f(X, Y)')
mgu = term.unify(Term.parse_term('f(Y, X)'))
{var: str(mgu[var]) for var in mgu}

term = Term.parse_term('f(X1, h(X1), X2)')
mgu = term.unify(Term.parse_term('f(g(X3), X4, X3)'))
{var: str(mgu[var]) for var in mgu}

term = Term.parse_term('f(X1 ,g(X2, X3), X2, b)')
mgu = term.unify(Term.parse_term('f(g(h(a, X5), X2), X1, h(a, X4), X4)'))
{var: str(mgu[var]) for var in mgu}

We have all functions and methods we need to build and operate on atoms and terms. We can now further compose:

* conjunctions of atoms as rule bodies;
* rule heads (atoms) and rule bodies as rules;
* sequences of rules as logic programs.

We define a new class for each: `Conjunction`, `Rule`, and `LogicProgram`.

A `Conjunction` object is intended to be a list of `Atom` objects; this is achieved by letting `Conjunction` inherit from `list`. We can then define in `Conjunction` a method, `predicate_and_function_symbols()`, meant to collect all predicate and function symbols that occur in the atoms that make up a conjunction, while checking that they are all used in a consistent manner:

* no predicate symbol and no function symbol should be used with different arities in different atoms;
* no symbol should be used as a predicate symbol in an atom and as a function symbol in another atom.

The implementation of `predicate_and_function_symbols()` in `Conjunction` is similar to the implementation of `collected_symbols()` in `Expression`, with at their hearts uses of `consistently_add_to()` and `consistently_merge_to()`:

In [ ]:
class Conjunction(list):
    class ConjunctionError(Exception):
        pass

    def __init__(self, conjuncts):
        super().__init__(conjuncts)

    def predicate_and_function_symbols(self):
        predicate_symbols = {}
        function_symbols = {}
        for atom in self:
            atom_predicate_symbol, atom_function_symbols =\
                    atom.predicate_and_function_symbols()
            if not consistently_add_to(atom_predicate_symbol, predicate_symbols
                                      ):
                raise Conjunction.ConjunctionError(f'{self}: predicate symbol '
                                                   'used with many arities'
                                                  )
            if not consistently_merge_to(atom_function_symbols,
                                         function_symbols
                                        ):
                raise Conjunction.ConjunctionError(f'{self}: function symbol '
                                                   'used with many arities'
                                                  )
        if predicate_symbols.keys() & function_symbols.keys():
            raise Conjunction.ConjunctionError(f'{self}: symbol used as '
                                               'predicate and function symbol'
                                              )
        return predicate_symbols, function_symbols

In the following cell, we implement the `Rule` class, with three attributes: `head`, meant to denote an `Atom` object, `body`, meant to denote a `Conjunction` object, and `variables`, meant to denote the set of variables occurring in the atoms that make up a rule. `Conjunction` has a `predicate_and_function_symbols()` method; `Rule` also has a method with that name, similarly implemented, meant to collect all predicate and function symbols that occur in a rule, while checking that:

* neither the predicate symbol nor any function symbol in the rule's head is used in the rule's body with a different arity;
* no symbol is used as a predicate symbol in the rule's head and as a function symbol in the rule's body, and the other way around.

We implement in `Rule` the `__str__()` method to display a rule, properly formatted. Finally, we implement a function, `parse_rule()`, similar to the `parse_term()` and `parse_atom()` functions of the `Term` and `Atom` classes, respectively.

In [ ]:
class Rule:
    class RuleError(Exception):
        pass

    def __init__(self, head, body=[]):
        self.head = head
        self.body = body
        self.variables =  self.head.variables()
        for atom in self.body:
            self.variables |= atom.variables() 

    def __str__(self):
        if not self.body:
            return self.head.__str__() + '.'
        return ' :- '.join((self.head.__str__(),
                            ', '.join(atom.__str__() for atom in self.body)
                           )
                          ) + '.'

    def predicate_and_function_symbols(self):
        head_predicate_symbol, head_function_symbols =\
                self.head.predicate_and_function_symbols()
        if not self.body:
            return {head_predicate_symbol[0]: head_predicate_symbol[1]},\
                   head_function_symbols
        predicate_symbols, function_symbols =\
                self.body.predicate_and_function_symbols()        
        if not consistently_add_to(head_predicate_symbol, predicate_symbols):
            raise Rule.RuleError(f"{self}: head's predicate symbol used with "
                                 'different arity in body'
                                )
        if not consistently_merge_to(head_function_symbols, function_symbols):
            raise Rule.RuleError(f'{self}: function symbol used with '
                                 'different arities in head and body'
                                )
        if predicate_symbols.keys() & function_symbols.keys():
            raise Rule.RuleError(f'{self}: symbol used as predicate and '
                                 'function symbols, one in head, the other in '
                                 'body'
                                )
        return predicate_symbols, function_symbols

    def parse_rule(expression):
        if expression[-1] != '.':
            raise Rule.RuleError(f'{expression}: does not end in a full stop')
        rule = expression[: -1].split(':-')
        if not (1 <= len(rule) <= 2):
            raise Rule.RuleError(f'{expression}: syntactically invalid')
        head = Expression.parse_item(rule[0])
        if not head:
            raise Rule.RuleError(f'{expression}: syntactically incorrect head')
        if len(rule) == 1:
            rule = Rule(head)
        else:
            body = Expression.parse_item(rule[1], parse_single_subitem=False)
            if not body:
                raise Rule.RuleError(f'{expression}: syntactically incorrect '
                                     'body'
                                    )
            rule = Rule(head, Conjunction(body))
        rule.predicate_and_function_symbols()
        return rule

In [ ]:
print(Rule.parse_rule('female(alice).'))

print(Rule.parse_rule('sisterof(X, Y) :- parents(X, M, F), female(X),'
                      ' parents(Y, M, F).'
                     )
     )

In the following cell, we implement that part of the `LogicProgram` class that creates a logic program object from the contents of a file, with three attributes: `program`, meant to denote a list of `Rule` objects, and `predicate_symbols` and `function_symbols`, meant to denote the sets of predicate and function symbols, respectively, occurring in the rules that make up the logic program. The `__init__()` method of `LogicProgram` computes `predicate_symbols` and `function_symbols` in a way that is similar to the implementation of the `predicate_and_function_symbols()` method of the `Conjunction` and `Rule` classes, while checking that:

* no predicate symbol or function symbol is used with different arities in different rules;
* no symbol is used both as a predicate symbol and as a function symbol in different rules.

Prolog comments start with `%`; lines in the file whose first nonspace symbol is `%`, as well as blank lines, are ignored.

The `individualise_underscores()` method of the `Expression` class is used to, for each rule, replace every occurrence of `_` in the rule as a new, unique variable:

In [ ]:
class LogicProgram:
    class LogicProgramError(Exception): 
        pass

    def __init__(self, filename):
        self.program = []
        self.predicate_symbols = {}
        self.function_symbols = {}
        rule_nb = 0
        with open(filename) as file:
            for rule in file:
                rule = rule.strip()
                if not rule or rule.startswith('%'):
                    continue
                rule = Rule.parse_rule(rule)
                rule_nb += 1
                rule_predicate_symbols, rule_function_symbols =\
                        rule.predicate_and_function_symbols()
                if not consistently_merge_to(rule_predicate_symbols,
                                             self.predicate_symbols
                                            )\
                   or not consistently_merge_to(rule_function_symbols,
                                                self.function_symbols
                                               ):
                    raise LogicProgram.LogicProgramError(
                             f'Symbol arity in rule nb {rule_nb} '
                            'inconsistent with arities in previous rules'
                                                        )
                rule_variables = rule.head.variables()
                for atom in rule.body:
                    rule_variables |= atom.variables()
                underscore_index =\
                      rule.head.individualise_underscores(rule_variables)
                for atom in rule.body:
                    underscore_index =\
                            atom.individualise_underscores(rule_variables,
                                                           underscore_index
                                                          )
                self.program.append(rule)
        if self.predicate_symbols.keys() & self.function_symbols.keys():
            raise LogicProgram.LogicProgramError('Symbol used as predicate '
                                                 'and function symbols'
                                                )

Everything is now in place for the Prolog interpreter to solve queries, with an exploration of the search tree that defaults to depth-first, but that can be changed to breadth-first. The following tracing function, `trace_solve()`, is meant to explain and illustrate the workings of the interpreter. The second argument, `query`, is meant to be a string that represents a single atom or a sequence of atoms separated by spaces, to be interpreted as a goal or an implicitly conjuncted sequence of goals; the `parse_item()` function from the `Expression` class, with the argument `parse_single_subitem` set to `False`, is all we need to convert `query` to a list of `Atom` elements, one for each goal. The only place where depth-first and breadth-first explorations differ is at the very end of the function:

In [ ]:
def trace_solve(logic_program, query, depth_first=True):
    query = Conjunction(Expression.parse_item(query,
                                              parse_single_subitem=False
                                             )
                       )
    query_variables = {var for atom in query for var in atom.variables()}
    # A list of pairs consisting of:
    # - a list of goals to be solved, and
    # - the substitution to apply to the variables that occur in the
    #   query as determined by the unifications computed so far.
    goals_solution_pairs =\
            deque([(deque(query), {var: Term(var) for var in query_variables})
                  ]
                 )
    while goals_solution_pairs:
        goals, solution = goals_solution_pairs.popleft()
        print('\nDealing with following goals and partial solution:')
        print('   ', ', '.join(goal.__str__() for goal in goals), end='')
        print('   ', ', '.join(f'{var} -> {solution[var]}'
                                    for var in solution)
             )
        if not goals:
            print('    No goal left, solution is complete')
            yield {var: solution[var].__str__() for var in solution}
            continue
        reserved_variables =\
           query_variables |{var for atom in goals for var in atom.variables()}
        goal = goals.popleft()
        next_goals_solution_pairs = deque()
        for rule in logic_program.program:
            variable_renaming = Expression.fresh_variables(rule.variables,
                                                           reserved_variables
                                                          )
            head = rule.head.rename_variables(variable_renaming)
            mgu = goal.unify(head)
            if mgu is not None:
                print('    First goal unified with head of rule:', rule)
                print('      Renaming variables, head becomes:', head)
                print('      Mgu:',
                      ', '.join(f'{var} -> {mgu[var]}' for var in mgu)
                     )
                new_goals =\
                    deque(atom.rename_variables(variable_renaming)\
                              .substitute(mgu) for atom in rule.body
                         )
                new_goals.extend(goal.substitute_in_copy(mgu) for goal in goals
                                )
                if new_goals:
                    print('      New goals to solve: ', end='')
                    print(', '.join(goal.__str__() for goal in new_goals))
                    print('      New partial solution:',
                          ', '.join(f'{var} -> '
                                    f'{solution[var].substitute_in_copy(mgu)}'
                                            for var in solution
                                   )
                         )

                next_goals_solution_pairs.append(
                                   (new_goals,
                                    {var: solution[var].substitute_in_copy(mgu)
                                         for var in solution
                                    }
                                   )
                                                )
        if depth_first:
            goals_solution_pairs.extendleft(reversed(next_goals_solution_pairs)
                                           )
        else:
            goals_solution_pairs.extend(next_goals_solution_pairs)

In [ ]:
cat prolog_ex_2.pl

Tracing execution for a closed goal, so checking that the goal is a logical consequence of the logic program:

In [ ]:
logic_program = LogicProgram('prolog_ex_2.pl')
for _ in trace_solve(logic_program, 'grandparent(john, jack)'):
    pass

Tracing the depth-first search exploration for solutions to the goal `grandparent(john, X)` as illustrated with the first tree above:

In [ ]:
logic_program = LogicProgram('prolog_ex_2.pl')
for _ in trace_solve(logic_program, 'grandparent(john, X)'):
    pass

And the same but changing exploration to breadth-first:

In [ ]:
logic_program = LogicProgram('prolog_ex_2.pl')
for _ in trace_solve(logic_program, 'grandparent(john, X)', False):
    pass

In [ ]:
cat prolog_ex_5.pl

Tracing the depth-first search exploration for solutions to the goal `join(X, X, Y)` as illustrated with the third tree above yields the following. As there are infinitely many solutions, we use the `islice` class from the `itertools` module to trace the search for the first 3 solutions only: 

In [ ]:
logic_program = LogicProgram('prolog_ex_5.pl')
for _ in islice(trace_solve(logic_program, 'join(X, X, Y)'), 3):
    pass

The implementation of the Prolog interpreter in `LogicProgram` should now be clear. The only addition to the code of the tracing function is that we check that the goals to solve are built from predicate and function symbols that all occur in the logic program:

In [ ]:
class LogicProgram(LogicProgram):
    def solve(self, query, depth_first=True):
        query = Conjunction(Expression.parse_item(query,
                                                  parse_single_subitem=False
                                                 )
                           )
        query_predicate_symbols, query_function_symbols =\
                query.predicate_and_function_symbols()
        if any(predicate_symbol not in self.predicate_symbols
               or query_predicate_symbols[predicate_symbol]
               != self.predicate_symbols[predicate_symbol]
                    for predicate_symbol in query_predicate_symbols
              ):
            raise LogicProgram.QuerryError(f'{query}: predicate symbol '
                                           'in query not in program'
                                          )
        if any(function_symbol not in self.function_symbols
               or query_function_symbols[function_symbol]
               != self.function_symbols[function_symbol]
                    for function_symbol in query_function_symbols
              ):
            raise LogicProgram.QuerryError(f'{query}: function symbol '
                                           'in query not in program'
                                          )               
        query_variables = {var for atom in query for var in atom.variables()}
        # A list of pairs consisting of:
        # - a list of goals to be solved, and
        # - the substitution to apply to the variables that occur in the
        #   query as determined by the unifications computed so far.
        goals_solution_pairs = deque([(deque(query),
                                       {var: Term(var)
                                            for var in query_variables
                                       }
                                      )
                                     ]
                                    )
        while goals_solution_pairs:
            goals, solution = goals_solution_pairs.popleft()
            if not goals:
                yield {var: solution[var].__str__() for var in solution}
                continue
            reserved_variables = query_variables\
                                 | {var for atom in goals
                                           for var in atom.variables()
                                   }
            goal = goals.popleft()
            next_goals_solution_pairs = deque()
            for rule in self.program:
                variable_renaming =\
                        Expression.fresh_variables(rule.variables,
                                                   reserved_variables
                                                  )
                head = rule.head.rename_variables(variable_renaming)
                mgu = goal.unify(head)
                if mgu is not None:
                    new_goals = deque(atom.rename_variables(variable_renaming)
                                         .substitute(mgu) for atom in rule.body
                                     )
                    new_goals.extend(goal.substitute_in_copy(mgu)
                                          for goal in goals
                                    )
                    next_goals_solution_pairs.append(
                                   (new_goals,
                                    {var: solution[var].substitute_in_copy(mgu)
                                         for var in solution
                                    }
                                   )
                                                    )
            if depth_first:
                goals_solution_pairs.extendleft(
                                        reversed(next_goals_solution_pairs)
                                               )
            else:
                goals_solution_pairs.extend(next_goals_solution_pairs)

In [ ]:
LP = LogicProgram('prolog_ex_2.pl')
for solution in LP.solve('grandparent(john, X)'):
        print('   ', solution)